In [1]:
import torch
import torch.nn as nn
import torch
import torch.optim as optim

from torchvision import datasets, transforms
from torch.utils.data import DataLoader

#Progress bar
from tqdm import tqdm


#For reading CSV and for math
import numpy as np
import pandas as pd

#Visualize some datasets
import matplotlib.pyplot as plt

#Check our work directory
import os
import shutil


In [2]:
photo_folder_path = r'train\train'
csv_file_path = r'train.csv'

In [3]:
df = pd.read_csv(csv_file_path)

In [4]:
train_blur = df[df["blur"]==1]
train_sharp = df[df["blur"]==0]

print("blur images:", len(train_blur))
print("sharp images:", len(train_sharp))

blur images: 1297
sharp images: 1367


In [5]:
train_val_split = 0.8  # 80% of the photos will be used for training, the rest will be used for validation

In [6]:
dataset_folder_path = r"Dataset"

In [7]:
import random

if not os.path.exists(dataset_folder_path):

    if not os.path.exists(os.path.join(dataset_folder_path, 'train', 'blur')):
        os.makedirs(os.path.join(dataset_folder_path, 'train', 'blur'))
    if not os.path.exists(os.path.join(dataset_folder_path, 'train', 'not_blur')):
        os.makedirs(os.path.join(dataset_folder_path, 'train', 'not_blur'))
    if not os.path.exists(os.path.join(dataset_folder_path, 'valid', 'blur')):
        os.makedirs(os.path.join(dataset_folder_path, 'valid', 'blur'))
    if not os.path.exists(os.path.join(dataset_folder_path, 'valid', 'not_blur')):
        os.makedirs(os.path.join(dataset_folder_path, 'valid', 'not_blur'))

    for _, row in df.iterrows():
        filename = row['filename']
        blur = row['blur']
        file_path = os.path.join(photo_folder_path, filename)
        if blur:
            # copy the file to the "train" folder 80% of the time and to the "valid" folder 20% of the time
            if random.random() < train_val_split:
                shutil.copy(file_path, os.path.join(dataset_folder_path, 'train', "blur"))
            else:
                shutil.copy(file_path, os.path.join(dataset_folder_path, 'valid', "blur"))
        else:
            # copy the file to the "train" folder 80% of the time and to the "valid" folder 20% of the time
            if random.random() < train_val_split:
                shutil.copy(file_path, os.path.join(dataset_folder_path, 'train', "not_blur"))
            else:
                shutil.copy(file_path, os.path.join(dataset_folder_path, 'valid', "not_blur"))


In [8]:
LR = 0.001 * 2
BATCH_SIZE = 4
EPOCHS = 10

# BEST PARAMS
#LR = 0.001
#EPOCHS = 15

In [9]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)

In [10]:
train_dir = 'Dataset/train'
val_dir = 'Dataset/valid'

In [11]:
train_folder = datasets.ImageFolder(root=train_dir, transform=transform)
val_folder = datasets.ImageFolder(root=val_dir,transform=transform)

In [12]:
train_folder.classes

['blur', 'not_blur']

In [13]:
train_loader = DataLoader(train_folder,
                          batch_size=BATCH_SIZE,
                          shuffle=True)
val_loader = DataLoader(val_folder,
                          batch_size=BATCH_SIZE,
                          shuffle=True)

In [14]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        # input [3, 640, 640]
        self.cnn = nn.Sequential(
            nn.Conv2d(in_channels = 3, out_channels = 64, kernel_size = 3, stride = 1, padding = 1),  # [64, 640, 640]
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2, padding = 0),      # [64, 320, 320]

            nn.Conv2d(in_channels = 64, out_channels = 128, kernel_size = 3, stride = 1, padding = 1), # [128, 320, 320]
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2, padding = 0),      # [128, 160, 160]

            nn.Conv2d(in_channels = 128, out_channels = 256, kernel_size = 3, stride = 1, padding = 1), # [256, 160, 160]
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2, padding = 0),      # [256, 80, 80]

            nn.Conv2d(in_channels = 256, out_channels = 512, kernel_size = 3, stride = 1, padding = 1), # [512, 80, 80]
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2, padding = 0),       # [512, 40, 40]
            
            nn.Conv2d(in_channels = 512, out_channels = 512, kernel_size = 3, stride = 1, padding = 1), # [512, 40, 40]
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2, padding = 0),       # [512, 20, 20]
        )
        self.fc = nn.Sequential(
            nn.Linear(in_features = 512*20*20, out_features = 1024),
            nn.ReLU(),
            nn.Linear(in_features = 1024, out_features = 512),
            nn.ReLU(),
            nn.Linear(in_features = 512, out_features = 128), 
            nn.ReLU(), 
            nn.Linear(in_features = 128, out_features = 2)
        )

    def forward(self, x):
        out = self.cnn(x)
        out = out.view(out.size()[0], -1)
        return self.fc(out)
net = Net()

In [15]:
for p in net.parameters():
    print(p.shape)

torch.Size([64, 3, 3, 3])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([128, 64, 3, 3])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([256, 128, 3, 3])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([512, 256, 3, 3])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512, 512, 3, 3])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([1024, 204800])
torch.Size([1024])
torch.Size([512, 1024])
torch.Size([512])
torch.Size([128, 512])
torch.Size([128])
torch.Size([2, 128])
torch.Size([2])


In [16]:
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
print('Using: ',device)
net.to(device)

Using:  cuda


Net(
  (cnn): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (12): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (14

In [17]:
print("total allocated memory on gpu (pytorch):", torch.cuda.memory.memory_allocated(device))

total allocated memory on gpu (pytorch): 856899072


In [18]:
# Cross Entropy Loss  
error = nn.CrossEntropyLoss()

#Adam optimizer
optimizer = optim.Adam(net.parameters(), lr = LR)

In [20]:
from sklearn.metrics import roc_auc_score

for epoch in range(EPOCHS):
    train_metric = np.array([])
    val_metric = np.array([])
    train_loss = np.array([])
    val_loss = np.array([])

    loop = tqdm(train_loader, leave=True)
    net.train()
    for batch in loop:
        optimizer.zero_grad()
        inputs = batch[0].to(device)
        labels = batch[1].to(device)
        y_pred = net(inputs)

        loss = error(y_pred,labels.long())
        loss.backward()

        loop.set_description(f"Epoch {epoch+1}")
        optimizer.step()
        try:
            auc = roc_auc_score(labels.cpu().numpy(), np.argmax(y_pred.cpu().data.numpy(), axis=1))     # use AUC-ROC metric if different class are represented
            train_metric = np.append(train_metric, auc)
        except:
            train_metric = np.append(train_metric, np.sum(np.argmax(y_pred.cpu().data.numpy(),axis=1) == labels.cpu().numpy()) / BATCH_SIZE)        # else use Accuracy metric
        train_loss = np.append(train_loss, loss.item())
        loop.set_postfix(loss=np.mean(train_loss), AUC_ROC=np.mean(train_metric))
    net.eval()
    with torch.no_grad():
        loop = tqdm(val_loader, leave=True)
        for batch in loop:
            inputs = batch[0].to(device)
            labels = batch[1].to(device)
            val_pred = net(inputs)

            batch_loss = error(val_pred,labels.long())
            
            loop.set_description(f"Validation {epoch+1}")
            try:
                auc = roc_auc_score(labels.cpu().numpy(), np.argmax(val_pred.cpu().data.numpy(), axis=1))     # use AUC-ROC metric if different class are represented
                val_metric = np.append(val_metric, auc)
            except:
                val_metric = np.append(val_metric, np.sum(np.argmax(val_pred.cpu().data.numpy(),axis=1) == labels.cpu().numpy()) / BATCH_SIZE)        # else use Accuracy metric
            val_loss = np.append(val_loss, loss.item())
            loop.set_postfix(loss=np.mean(val_loss), AUC_ROC=np.mean(val_metric))

Epoch 1:   1%|          | 3/541 [00:01<04:10,  2.15it/s, AUC_ROC=0.5, loss=40.9]

[0 0 0 0] [1 1 1 1]


Epoch 1:   1%|▏         | 8/541 [00:03<03:38,  2.44it/s, AUC_ROC=0.5, loss=22]  

[1 1 1 1] [0 0 0 0]


Epoch 1:   2%|▏         | 9/541 [00:03<03:37,  2.45it/s, AUC_ROC=0.5, loss=19.7]

...

Epoch 1:  84%|████████▎ | 453/541 [02:59<00:34,  2.55it/s, AUC_ROC=0.512, loss=1.21]

[1 1 1 1] [1 1 1 1]


Epoch 1:  84%|████████▍ | 454/541 [02:59<00:34,  2.53it/s, AUC_ROC=0.512, loss=1.21]

[0 0 0 0] [1 1 1 1]


Epoch 1:  86%|████████▌ | 466/541 [03:04<00:29,  2.56it/s, AUC_ROC=0.511, loss=1.2] 

In [ ]:
torch.save(net.state_dict(), rf"model lr{LR} e{EPOCHS}.pt")

In [ ]:
net.load_state_dict(torch.load("model lr0.001 e15.pt"))


In [ ]:
from torch.autograd import Variable
from PIL import Image

def predict_image(image_path, model, device):
    image = Image.open(image_path)

    transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
    )

    image_tensor = transform(image).float()
    image_tensor = image_tensor.unsqueeze(0)

    if torch.cuda.is_available() and device == "cuda":
        image_tensor.cuda()
        input = Variable(image_tensor.cuda(), requires_grad=True) #torch.Tensor(image_tensor.cuda(), requires_grad=True)
    else:
        input = Variable(image_tensor.cuda(), requires_grad=True) #torch.Tensor(image_tensor.cuda(), requires_grad=True)
    output = model(input)
    #label = output.data.cpu().numpy()
    label = torch.nn.functional.softmax(output.data.cpu())
    label = label.numpy()
    
    return label[0][0].round(1)
    #plt.imshow(image)


In [ ]:
predict_image(r"C:\Users\PC\Desktop\Проги\ML\Задание на стажировку\test\test\zvavwunqimpeqmiahuxd.jpg", net, device)

In [ ]:
path = r"test\test"
test_imgs = []
for file in os.listdir(path):
    test_imgs.append({'filename': file})
df_test = pd.DataFrame(test_imgs)
print("Test images: ", len(df_test))

In [ ]:
blurness = []
for file in os.listdir(path):
    blurness.append(predict_image(os.path.join(path, file), net, device))


In [ ]:
blurness

In [ ]:
df_test['blur'] = blurness
df_test['filename'] = df_test['filename'].apply(lambda x : os.path.split(x)[1])
df_test.to_csv('submission 2.csv', index=False)